In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [ ]:
# Import relevant libraries
import nltk

from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

nltk.download('wordnet')

import re, string, random
import csv


In [ ]:
# define function remove noise, this function takes a list of tokens and removes words
# that don't contribute to the overall sentiment of the text, connecting and filler words.
# create preprocess_text function
def remove_noise(text):

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    mystem = SnowballStemmer("english")

    # stem the tokens
    stemmed_tokens = [mystem.stem(token) for token in filtered_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(stemmed_tokens)

    return processed_text



In [ ]:
# import the training data set
df = pd.read_csv("/kaggle/input/amazon-customer-reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# define stop_words to remove from reviews
stop_words = stopwords.words('english')

# remove noise from text reviews
df['reviews.text'] = df['reviews.text'].apply(remove_noise)

# seperate the df into positive and negative reviews 
pos_rev =  df.loc[df['reviews.rating'] > 2] 
neg_rev = df.loc[df['reviews.rating'] <= 2] 
text = df['reviews.text']

In [ ]:
# Create negative and positive dataset
positive_dataset = pd.DataFrame()
negative_dataset = pd.DataFrame()

# load dataset with positive reviews and attached positive string to each
positive_dataset['reviews.text'] = pos_rev['reviews.text']
positive_dataset['sentiment'] = 'Positive'

# load dataset with negative reviews and attached negative string to each
negative_dataset['reviews.text'] = neg_rev['reviews.text']
negative_dataset['sentiment'] = 'Negative'

print(negative_dataset.head())

In [ ]:
# create main data set
dataset = pd.concat([positive_dataset, negative_dataset])

# Shuffle the data
dataset = dataset.sample(frac = 1)

# initialize NLTK sentiment analyzer

analyzer = SentimentIntensityAnalyzer()

# create get_sentiment function

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)
    if scores['pos'] > 0:
        sentiment = 'positive'
    else:
        sentiment = 'negative'

    return sentiment


# apply get_sentiment function
dataset['sentiment'] = dataset['reviews.text'].apply(get_sentiment)

# write out the dataset to csv
dataset.to_csv("predicted_sentiment.csv", index=False)

